In [15]:
import sys
sys.path.append("..")
import torch
from torch import nn
from torch import optim
import torchvision.transforms as transforms
import time
import os
# convolution patch embedding
# from Tensorized_components.patch_embedding  import Patch_Embedding        
from Tensorized_components.tcl_patch_embedding  import  PatchEmbedding  as  Patch_Embedding      
from Tensorized_components.w_msa_w_o_b  import WindowMSA     
from Tensorized_components.sh_wmsa_w_o_b import ShiftedWindowMSA     
from Tensorized_components.patch_merging  import TensorizedPatchMerging  
from Tensorized_Layers.TCL  import TCL  as TCL_CHANGED   
from Tensorized_Layers.TRL import TRL   
from Utils.Accuracy_measures import topk_accuracy
from Utils.cifar100_loaders import get_cifar100_dataloaders
from Utils.Num_parameter import count_parameters
import argparse
import numpy as np
import random
from torchvision.transforms import RandAugment, RandomErasing
from torch.optim.lr_scheduler import CosineAnnealingLR




In [16]:
class SwinBlock1(nn.Module):
    """
    A class representing 'Block 1' in your Swin Transformer.
    This captures the sequence of:
        (1) Window MSA + residual
        (2) TCL + residual
        (3) Shifted Window MSA + residual
        (4) TCL + residual
    but only for the first block’s hyperparameters and submodules.
    """
    def __init__(self, w_msa, sw_msa, tcl1,tcl2,tcl3,tcl4, embed_shape=(3,4,4), dropout=0):
        super(SwinBlock1, self).__init__()
        # Typically each sub-layer has its own LayerNorm
        self.norm1 = nn.LayerNorm(embed_shape)
        self.norm2 = nn.LayerNorm(embed_shape)
        self.norm3 = nn.LayerNorm(embed_shape)
        self.norm4 = nn.LayerNorm(embed_shape)

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # We pass in pre-built modules (WindowMSA, ShiftedWindowMSA, TCL)
        self.w_msa = w_msa
        self.sw_msa = sw_msa
        self.tcl1 = tcl1
        self.gelu = nn.GELU()
        self.tcl2 = tcl2
        self.tcl3 = tcl3
        self.tcl4 = tcl4
    def forward(self, x):
        # ----- First Window MSA + Residual -----
        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.w_msa(x))
        x = x + x_res

        # ----- TCL + Residual -----
        x_res = x
        x = self.norm2(x)
        x = self.tcl1(x)
        x = self.gelu(x)
        x = self.tcl2(x)
        x = x + x_res

        # ----- Shifted Window MSA + Residual -----
        x_res = x
        x = self.norm3(x)
        x = self.dropout(self.sw_msa(x))
        x = x + x_res

        # ----- TCL + Residual -----
        x_res = x
        x = self.norm4(x)
        x = self.tcl3(x)
        x = self.gelu(x)
        x = self.tcl4(x)
        x = x + x_res

        return x


In [17]:
class SwinBlock2(nn.Module):
    def __init__(self, w_msa, sw_msa, tcl1, tcl2 , tcl3 , tcl4,  embed_shape=(3,4,8), dropout=0):
        super(SwinBlock2, self).__init__()
        # LN layers
        self.norm1 = nn.LayerNorm(embed_shape)
        self.norm2 = nn.LayerNorm(embed_shape)
        self.norm3 = nn.LayerNorm(embed_shape)
        self.norm4 = nn.LayerNorm(embed_shape)

        self.dropout = nn.Dropout(dropout)

        self.w_msa = w_msa
        self.sw_msa = sw_msa
        self.tcl1 = tcl1
        self.tcl2 = tcl2
        self.tcl3 = tcl3
        self.tcl4 = tcl4
        self.gelu = nn.GELU()


    def forward(self, x):
        # Window MSA
        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.w_msa(x))
        x = x + x_res

        # TCL
        x_res = x
        x = self.norm2(x)
        x = self.tcl1(x)
        x = self.gelu(x)
        x = self.tcl2(x)
        x = x + x_res

        # Shifted Window MSA
        x_res = x
        x = self.norm3(x)
        x = self.dropout(self.sw_msa(x))
        x = x + x_res

        # TCL
        x_res = x
        x = self.norm4(x)
        x = self.tcl3(x)
        x = self.gelu(x)
        x = self.tcl4(x)
        x = x + x_res

        return x


In [18]:
class SwinTransformer(nn.Module):
    def __init__(self,
                 img_size=32,
                 patch_size=4,
                 in_chans=3,
                 embed_shape=(3,4,4),
                 bias=True,
                 dropout=0,
                 device="cuda"):
        super(SwinTransformer, self).__init__()

        self.device = device

# tcl patch embedding 


# TODO : change batch size and device to cuda
        self.patch_embedding = Patch_Embedding(
            input_size=(16,3,32,32),
            patch_size=patch_size,
            embed_dim=embed_shape,
            bias=bias,
            device="cpu",
            ignore_modes = (0,1,2)
        )
# convolution 
        # self.patch_embedding = Patch_Embedding(
        #     img_size=img_size,
        #     patch_size=patch_size,
        #     in_chans=in_chans,
        #     embed_shape=embed_shape,
        #     bias=bias
        # )

        # -------------------------------- block 1 --------------------------

        self.w_msa_1 = WindowMSA(
            window_size=4,
            embed_dims=embed_shape,
            rank_window=embed_shape,
            head_factors=(1,2,2),
            device=self.device
        )

        self.sw_msa_1 = ShiftedWindowMSA(
            window_size=4,
            embed_dims=embed_shape,
            rank_window=embed_shape,
            head_factors=(1,2,2),
            device=self.device
        )

        self.tcl_1 = TCL_CHANGED(
            input_size=(16, 8, 8, 3,4,4),
            rank=(3,4,16),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        self.tcl_1_2 = TCL_CHANGED(
            input_size=(16, 8, 8, 3,4,16),
            rank=(3,4,4),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        self.tcl_1_3 =  TCL_CHANGED(
            input_size=(16, 8, 8, 3,4,4),
            rank=(3,4,16),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )


        self.tcl_1_4 = TCL_CHANGED(
            input_size=(16, 8, 8, 3,4,16),
            rank=(3,4,4),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )


        self.block1_list = nn.ModuleList([
            SwinBlock1(
                w_msa=self.w_msa_1,
                sw_msa=self.sw_msa_1,
                tcl1=self.tcl_1,
                tcl2 = self.tcl_1_2,
                tcl3 = self.tcl_1_3,
                tcl4 = self.tcl_1_4,
                embed_shape=embed_shape,
                dropout=dropout
            )
            for _ in range(2)
        ])

        # -------------------------------- block 2 --------------------------


        self.patch_merging_1 = TensorizedPatchMerging(
            input_size=(16, 8, 8, 3,4,4),
            in_embed_shape=embed_shape,
            out_embed_shape=(3,4,8),
            bias=bias,
            ignore_modes=(0, 1, 2),
            device=self.device
        )

        self.w_msa_2 = WindowMSA(
            window_size=4,
            embed_dims=(3,4,8),
            rank_window=(3,4,8),
            head_factors=(1,2,4),
            device=self.device
        )

        self.sw_msa_2 = ShiftedWindowMSA(
            window_size=4,
            embed_dims=(3,4,8),
            rank_window=(3,4,8),
            head_factors=(1,2,4),
            device=self.device
        )

        self.tcl_2 = TCL_CHANGED(
            input_size=(16, 4, 4, 3,4,8),
            rank=(3,4,32),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )




        self.tcl_2_2 = TCL_CHANGED(
            input_size=(16, 4, 4, 3,4,32),
            rank=(3,4,8),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        self.tcl_2_3 = TCL_CHANGED(
            input_size=(16, 4, 4, 3,4,8),
            rank=(3,4,32),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        
        self.tcl_2_4 = TCL_CHANGED(
            input_size=(16, 4, 4, 3,4,32),
            rank=(3,4,8),
            ignore_modes=(0, 1, 2),
            bias=bias,
            device=self.device
        )

        # We repeat Block2 two times
        self.block2_list = nn.ModuleList([
            SwinBlock2(
                w_msa=self.w_msa_2,
                sw_msa=self.sw_msa_2,
                tcl1=self.tcl_2,
                tcl2 = self.tcl_2_2,
                tcl3 = self.tcl_2_3,
                tcl4 = self.tcl_2_4,
                embed_shape=(3,4,8),  
                dropout=dropout
            )
            for _ in range(2)
        ])


        # -------------------------------- classifier --------------------------

    

        self.classifier = TRL(input_size=(16,3,4,8),
                            output=(200,),
                            rank=(3,4,8,200),
                            ignore_modes=(0,),
                            bias=bias,
                            device=self.device) 
        

        # positoin embedding


        self.pos_embedding = nn.Parameter(
            torch.randn(1,
                        8,
                        8,
                        3,
                        4,
                        4,
                        device = self.device
                        ), requires_grad=True)

    def forward(self, x):
 

        x = self.patch_embedding(x)

        x += self.pos_embedding

        for i, blk in enumerate(self.block1_list, 1):
            x = blk(x)

        x = self.patch_merging_1(x)

        for i, blk in enumerate(self.block2_list, 1):
            x = blk(x)

        x = x.mean(dim=(1, 2))

        output = self.classifier(x)
        return output

In [19]:


def set_seed(seed: int = 42):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)  
    torch.cuda.manual_seed(seed)  
    torch.cuda.manual_seed_all(seed)  
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False


def mixup_data(x, y, alpha=0.8):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


def get_cosine_schedule_with_warmup(optimizer, num_warmup_epochs, num_training_epochs):
    def lr_lambda(epoch):
        if epoch < num_warmup_epochs:
            return float(epoch) / float(max(1, num_warmup_epochs))
        return 0.5 * (1. + np.cos(np.pi * (epoch - num_warmup_epochs) / (num_training_epochs - num_warmup_epochs)))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [20]:
# Setup the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f'Device is set to : {device}')


SEED = None 

if SEED is None:
    print(f'No seed is set!')
else:
    set_seed(seed=SEED)


# Configs

TEST_ID = 'Test_ID00091'
batch_size = 256
n_epoch = 400
image_size = 32

model = SwinTransformer(img_size=32,patch_size=4,in_chans=3,embed_shape=(3,4,4),bias=True,device=device).to(device)

transform_train = transforms.Compose([
            RandAugment(), 
            transforms.RandomHorizontalFlip(),
            transforms.RandomResizedCrop(image_size, scale=(0.8, 1.0)),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            RandomErasing(p=0.25)
        ])
transform_test = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])


# train_size = 4
train_loader, _ = get_cifar100_dataloaders('../datasets', transform_train, transform_test, batch_size, image_size, train_size, repeat_count=5)


Device is set to : cpu
No seed is set!


In [21]:
num_parameters = count_parameters(model)
print(f'This Model has {num_parameters} parameters')
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


# Define train and test functions (use examples)
def train_epoch(loader, epoch):
    model.train()

    start_time = time.time()
    running_loss = 0.0
    correct = {1:0.0, 2:0.0, 3:0.0, 4:0.0, 5:0.0} # set the initial correct count for top1-to-top5 accuracy

    for i, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
    
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
    
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        accuracies = topk_accuracy(outputs, targets, topk=(1, 2, 3, 4, 5))
        for k in accuracies:
            correct[k] += accuracies[k]['correct']
        # print(f'batch{i} done!')

    elapsed_time = time.time() - start_time
    top1_acc, top2_acc, top3_acc, top4_acc, top5_acc = [(correct[k]/len(loader.dataset)) for k in correct]
    avg_loss = running_loss / len(loader.dataset)

    report_train = f'Train epoch {epoch}: top1={top1_acc}%, top2={top2_acc}%, top3={top3_acc}%, top4={top4_acc}%, top5={top5_acc}%, loss={avg_loss}, time={elapsed_time}s'
    print(report_train)

    return report_train

def test_epoch(loader, epoch):
    model.eval()

    start_time = time.time()
    running_loss = 0.0
    correct = {1:0.0, 2:0.0, 3:0.0, 4:0.0, 5:0.0} # set the initial correct count for top1-to-top5 accuracy

    for _, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        running_loss += loss.item()
        accuracies = topk_accuracy(outputs, targets, topk=(1, 2, 3, 4, 5))
        for k in accuracies:
            correct[k] += accuracies[k]['correct']

    elapsed_time = time.time() - start_time
    top1_acc, top2_acc, top3_acc, top4_acc, top5_acc = [(correct[k]/len(loader.dataset)) for k in correct]
    avg_loss = running_loss / len(loader.dataset)

    report_test = f'Test epoch {epoch}: top1={top1_acc}%, top2={top2_acc}%, top3={top3_acc}%, top4={top4_acc}%, top5={top5_acc}%, loss={avg_loss}, time={elapsed_time}s'
    print(report_test)

    return report_test

# Set up the directories to save the results
result_dir = os.path.join('../results', TEST_ID)
result_subdir = os.path.join(result_dir, 'accuracy_stats')
model_subdir = os.path.join(result_dir, 'model_stats')

os.makedirs(result_subdir, exist_ok=True)
os.makedirs(model_subdir, exist_ok=True)

with open(os.path.join(result_dir, 'model_stats', 'model_info.txt'), 'a') as f:
    f.write(f'total number of parameters:\n{num_parameters}')

# Train from Scratch - Just Train
print(f'Training for {len(range(n_epoch))} epochs\n')
for epoch in range(0+1,n_epoch+1):
    report_train = train_epoch(train_loader, epoch)
    # report_test = test_epoch(test_loader, epoch)

    report = report_train + '\n' #+ report_test + '\n\n'
    if epoch % 5 == 0:
        model_path = os.path.join(result_dir, 'model_stats', f'Model_epoch_{epoch}.pth')
        torch.save(model.state_dict(), model_path)
    with open(os.path.join(result_dir, 'accuracy_stats', 'report_train.txt'), 'a') as f:
        f.write(report)
            

This Model has 72391 parameters
Training for 400 epochs

Train epoch 1: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.273856520652771, time=0.2713048458099365s
Train epoch 2: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.2737565517425537, time=0.32888174057006836s
Train epoch 3: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.27365598678588865, time=0.2349529266357422s
Train epoch 4: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.2735543727874756, time=0.2446751594543457s
Train epoch 5: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.27345116138458253, time=0.21798014640808105s
Train epoch 6: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.2733457088470459, time=0.24177885055541992s
Train epoch 7: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.2732372283935547, time=0.21313762664794922s
Train epoch 8: top1=0.0%, top2=0.0%, top3=0.0%, top4=0.0%, top5=0.0%, loss=0.27312483787536623, ti

KeyboardInterrupt: 